In [25]:
prefectures = ['北海道',
        '青森県',
        '岩手県',
        '宮城県',
        '秋田県',
        '山形県',
        '福島県',
        '茨城県',
        '栃木県',
        '群馬県',
        '埼玉県',
        '千葉県',
        '東京都',
        '神奈川県',
        '新潟県',
        '富山県',
        '石川県',
        '福井県',
        '山梨県',
        '長野県',
        '岐阜県',
        '静岡県',
        '愛知県',
        '三重県',
        '滋賀県',
        '京都県',
        '大阪県',
        '兵庫県',
        '奈良県',
        '和歌山県',
        '鳥取県',
        '島根県',
        '岡山県',
        '広島県',
        '山口県',
        '徳島県',
        '香川県',
        '愛媛県',
        '高知県',
        '福岡県',
        '佐賀県',
        '長崎県',
        '熊本県',
        '大分県',
        '宮崎県',
        '鹿児島県',
        '沖縄県']

In [1]:
from tabula import read_pdf
df = read_pdf("G:/マイドライブ/akiyamalab/卒論/負担率総集編/data/土地情報センター_平均価格・対前年平均変動表.pdf", pages="all")

Error importing jpype dependencies. Fallback to subprocess.
No module named 'jpype'


In [39]:
import pandas as pd

for i in range(1, 58):
    if i == 1:
        merged_df = df[i]
    
    else:
        merged_df = pd.concat([merged_df, df[i]])
        
# 地区名カラムの欠損値を持つ行を削除
merged_df = merged_df.dropna(subset=['地区名'])

merged_df.rename(columns={'平均価格(円/m2)':'住宅地', '対前年平均変動率(%)':'宅地見込地', 'Unnamed: 0':'商業地', 'Unnamed: 1':'工業地', 'Unnamed: 2':'全用途'}, inplace=True)
merged_df = merged_df[['地区名', '住宅地', '宅地見込地', '商業地', '工業地', '全用途']]

In [75]:
# 都道府県ごとにデータフレームを分割するための辞書を初期化
df_dict = {prefecture: pd.DataFrame(columns=['地区名', '住宅地', '宅地見込地', '商業地', '工業地', '全用途']) for prefecture in prefectures}

# 現在の都道府県を追跡する変数
current_prefecture = None

# データフレームを行ごとにループして、都道府県に基づいてデータを分割
for index, row in merged_df.iterrows():
    # 都道府県名が行に含まれているか確認
    for prefecture in prefectures:
        if prefecture in row['地区名']:
            current_prefecture = prefecture
            break
    
    # 現在の都道府県に行を追加
    if current_prefecture:
        row_df = pd.DataFrame([row.values], columns=row.index)
        df_dict[current_prefecture] = pd.concat([df_dict[current_prefecture], row_df], ignore_index=True)
        

for index, (pref, data) in enumerate(df_dict.items()):
    land_price_path = f'G:/マイドライブ/akiyamalab/卒論/負担率総集編/data/土地情報センター/{index+1}.csv'
    data.to_csv(land_price_path, index=False, encoding='shift_jis')


In [ ]:
merged_df_path = 'G:/マイドライブ/akiyamalab/卒論/負担率総集編/data/土地情報センター/全国_土地価格.csv'
merged_df.to_csv(merged_df_path, index=False, encoding='shift_jis')